## 24.3 Dijkstra's algorithm

### 24.3-1

> Run Dijkstra's algorithm on the directed graph of Figure 24.2, first using vertex $s$ as the source and then using vertex $z$ as the source. In the style of Figure 24.6, show the $d$ and $\pi$ values and the vertices in set $S$ after each iteration of the __while__ loop.

### 24.3-2

> Give a simple example of a directed graph with negative-weight edges for which Dijkstra's algorithm produces incorrect answers. Why doesn't the proof of Theorem 24.6 go through when negative-weight edges are allowed?

### 24.3-3

> Suppose we change line 4 of Dijkstra's algorithm to the following. 
> 
> 4  __while__ $|Q| > 1$
> 
> This change causes the __while__ loop to execute $|V| - 1$ times instead of $|V|$ times. Is this proposed algorithm correct?

Correct.

### 24.3-4

> Professor Gaedel has written a program that he claims implements Dijkstra's algorithm. The program produces $v.d$ and $v.\pi$ for each vertex $v \in V$. Give an $O(V + E)$-time algorithm to check the output of the professor's program. It should determine whether the $d$ and $\pi$ attributes match those of some shortest-paths tree. You may assume that all edge weights are nonnegative.

Relax on the shortest path tree.

### 24.3-5

> Professor Newman thinks that he has worked out a simpler proof of correctness for Dijkstra's algorithm. He claims that Dijkstra's algorithm relaxes the edges of every shortest path in the graph in the order in which they appear on the path, and therefore the path-relaxation property applies to every vertex reachable from the source. Show that the professor is mistaken by constructing a directed graph for which Dijkstra's algorithm could relax the edges of a shortest path out of order.

### 24.3-6

> We are given a directed graph $G = (V, E)$ on which each edge $(u, v) \in E$ has an associated value $r(u, v)$, which is a real number in the range $0 \le r(u, v) \le 1$ that represents the reliability of a communication channel from vertex $u$ to vertex $v$. We interpret $r(u, v)$ as the probability that the channel from $u$ to $v$ will not fail, and we assume that these probabilities are independent. Give an efficient algorithm to find the most reliable path between two given vertices.

$w(u, v) = \lg r(u, v)$.

### 24.3-7

> Let $G = (V, E)$ be a weighted, directed graph with positive weight function $w: E \rightarrow \{ 1, 2, \dots, W \}$ for some positive integer $W$, and assume that no two vertices have the same shortest-path weights from source vertex $s$. Now suppose that we define an unweighted, directed graph $G' = (V \cup V', E')$ by replacing each edge $(u, v) \in E$ with $w(u, v)$ unit-weight edges in series. How many vertices does $G'$ have? Now suppose that we run a breadth-first search on $G'$. Show that the order in which the breadth-first search of $G'$ colors vertices in $V$ black is the same as the order in which Dijkstra's algorithm extracts the vertices of $V$ from the priority queue when it runs on $G$.

$V + \sum_{(u, v) \in E} w(u, v) - E$.

### 24.3-8

> Let $G = (V, E)$ be a weighted, directed graph with nonnegative weight function $w: E \rightarrow \{ 0, 1, \dots, W \}$ for some nonnegative integer $W$. Modify Dijkstra's algorithm to compute the shortest paths from a given source vertex s in $O(WV + E)$ time.

Use array to store vertices.

### 24.3-9

> Modify your algorithm from Exercise 24.3-8 to run in $O((V + E) \lg W)$ time. (Hint: How many distinct shortest-path estimates can there be in $V - S$ at any point in time?)

Heap.

### 24.3-10

> Suppose that we are given a weighted, directed graph $G = (V, E)$ in which edges that leave the source vertex $s$ may have negative weights, all other edge weights are nonnegative, and there are no negative-weight cycles. Argue that Dijkstra's algorithm correctly finds shortest paths from $s$ in this graph.

# Hw 

### DAG shortest path 

assumption : DAG 이어야한다. (directed acyclic graph) 

using :  Topological sort : O(V + E)

total running time : O(V + E)

DAG Algo 이후, v.pi 에는 path 정보가, v.d 에는 shortest weight 가 입력된다. 

In [1]:
# node로 구현 
class node: 
    
    def __init__(self, name):
        self.name = name
        self.d = 0 
        self.f = 0
        self.pi = 0  
        self.key = 0
        self.color = 'unknown'
        self.edge = {}
        
class Graph: 
    
    time = 0
    
    def __init__(self, graph_dict=None, w = None):
        if graph_dict == None:
            graph_dict = {}
        self.graph_dict = graph_dict
        
        # node object로 list 만듦
        self.nodes = []
        for n in list(self.graph_dict.keys()):
            self.nodes.append(node(n))
        
        # node object로 dictionary 만듦
        self.nm = {}
        for k in range(len(list(self.graph_dict.keys()))):
            self.nm[self.nodes[k].name] = self.nodes[k]
        
        # edge 를 w 에 의해 부여 
        for j in range(len(list(self.graph_dict.keys()))):  # j vertex
            for i in self.graph_dict[self.nodes[j].name]:      # i edge 
                self.nodes[j].edge[i] = w[self.nodes[j].name][i]             # allocate weight 
                
    
    # sort the edge of G.E into nondecreasing order by weight w 
    def sort_edge(self):
        import operator
        temp = []
    
        for v in list(self.graph_dict.keys()):
            for i in list(self.nm[v].edge):
                temp.append((v,i,self.nm[v].edge[i]))

        t = sorted(temp, key = operator.itemgetter(2), reverse = False)
        #print(t)

        srt = []
        for m in range(len(t)): 
            srt.append([t[m][0],t[m][1]])
        #print(srt,"sort 이후 edge")
        return srt            
                
    """ DFS Algorithm running time : Θ(|V|+ |E|), 왜냐면, white 인 경우만 DFS_vist을 하므로"""
    # except DFS_vist, T = Θ(|V|)
    def DFS(self, a = []): # a = [] 이면 순서를 dict.keys() 순으로, o.w list (a)  순으로 searching 
    
        # running time O(V + E)
    
        for u in list(self.graph_dict.keys()):   
            self.nm[u].color = 'white'
            self.nm[u].pi = None
        
        self.time = 0
    
        #print("Debug sorting ", a)
        if a == []:
            #print(self.graph_dict.keys()," 순으로 searching 하게 된다.")
            for u in list(self.graph_dict.keys()): 
                if self.nm[u].color == 'white':
                    self.DFS_visit(u)
        else:
            #print(a," 순으로 searching 하게 된다.")
            for u in a: 
                if self.nm[u].color == 'white':
                    self.DFS_visit(u)
    
    
    # each DFS_vist, T = Θ(|V|)    
    def DFS_visit(self,u):
        self.time  = self.time + 1 
        self.nm[u].d = self.time 
        self.nm[u].color = 'gray'
    
        #print(G.graph_dict[u] ," 순으로 vertex", u, " 의 adjoint list를 searching 하게 된다.")
        for v in list(self.graph_dict[u]):
            if self.nm[v].color == 'white':
                self.nm[v].pi  = u
                self.DFS_visit(v)
        self.nm[u].color = 'black'
        self.time = self.time + 1 
        self.nm[u].f = self.time    
    

    # graph 에서 모든 edge들의 우선순위를 지켜주는 linear ordering 을 찾아 주었다. 
    # DAG(directed Acyclic graph) 에서 사용 
    def Topological_sort(self,l = []):
        import operator
        print("Topological_sort사용")
        self.DFS(l)
    
        # node object로 dictionary 만듦a
        topo = {}
        for k in range(len(list(self.graph_dict.keys()))):
            topo[self.nodes[k].name] = self.nodes[k].f
    
        temp = sorted(topo.items(), key = operator.itemgetter(1), reverse = True)
    
        srt = []
        for m in range(0,len(temp)):
            srt.append(temp[m][0])    
        print(srt, "Topological sort 결과 ")
        return srt 
    
    """ single src shortest path operation """
    def initialize_single_source(self,s):
        for v in list(self.graph_dict.keys()):
            self.nm[v].d = float('inf')
            self.nm[v].pi = None
        self.nm[s].d = 0
        # T = O(V)

    def relax(self,u,v):
        if self.nm[v].d > self.nm[u].d + self.nm[u].edge[v]:
            self.nm[v].d = self.nm[u].d + self.nm[u].edge[v]
            self.nm[v].pi = self.nm[u].name
        # T = O(1)
        
    def showinfo(self):
        print(self.graph_dict)
        print(" node dictionary : ")
        print(self.nm)
        
        for v in list(G.graph_dict.keys()):
            print(">> [ ",v, " node Info] : ")
            print(v, ", edge: ", G.nm[v].edge)
            print(v, ", pi: ", G.nm[v].pi)
            print(v, ", d: ", G.nm[v].d)

In [2]:
def DAG_shortest_paths(G,s):
    topo = G.Topological_sort()         # O(V + E)
    G.initialize_single_source(s)       # O(V)    
    for u in topo:
        # each v ∈ G.adj[u]
        for v in list(G.graph_dict[u]): # O(E) 
            G.relax(u,v)
    # running time: O(V + E)

In [3]:
vertices = ['r','s','t','x','y','z']
edges = [{'t','s'},{'x','t'},{'x','y','z'},{'y','z'},{'z'},{}]
weights = [{'t':3,'s':5},{'x':6,'t':2},{'x':7,'y':4,'z':2},{'y':-1,'z':1},{'z':-2},{}]
g = {}
w = {}
k = 0
for v in (vertices):
    g[v] = edges[k]
    w[v] = weights[k]  
    k = k + 1
print(g)
print(w)
G = Graph(g,w)


{'x': {'y', 'z'}, 'r': {'s', 't'}, 's': {'x', 't'}, 'y': {'z'}, 't': {'x', 'y', 'z'}, 'z': {}}
{'x': {'y': -1, 'z': 1}, 'r': {'s': 5, 't': 3}, 's': {'x': 6, 't': 2}, 'y': {'z': -2}, 't': {'x': 7, 'y': 4, 'z': 2}, 'z': {}}


In [4]:
DAG_shortest_paths(G,'s')
G.showinfo()

Topological_sort사용
['r', 's', 't', 'x', 'y', 'z'] Topological sort 결과 
{'x': {'y', 'z'}, 'r': {'s', 't'}, 's': {'x', 't'}, 'y': {'z'}, 't': {'x', 'y', 'z'}, 'z': {}}
 node dictionary : 
{'x': <__main__.node object at 0x0000002144945AC8>, 'r': <__main__.node object at 0x0000002144945B00>, 's': <__main__.node object at 0x0000002144945B38>, 'y': <__main__.node object at 0x0000002144945B70>, 't': <__main__.node object at 0x0000002144945BA8>, 'z': <__main__.node object at 0x0000002144945BE0>}
>> [  x  node Info] : 
x , edge:  {'y': -1, 'z': 1}
x , pi:  s
x , d:  6
>> [  r  node Info] : 
r , edge:  {'s': 5, 't': 3}
r , pi:  None
r , d:  inf
>> [  s  node Info] : 
s , edge:  {'x': 6, 't': 2}
s , pi:  None
s , d:  0
>> [  y  node Info] : 
y , edge:  {'z': -2}
y , pi:  x
y , d:  5
>> [  t  node Info] : 
t , edge:  {'x': 7, 'y': 4, 'z': 2}
t , pi:  s
t , d:  2
>> [  z  node Info] : 
z , edge:  {}
z , pi:  y
z , d:  3
